<h1><center>Projekt Seminar Deep Learning</center></h1>
<h2><center>Deep Reinforcement Learning with ME-TRPO</center></h2>
<center>Tobias Papen</center>
<center><a href="https://github.com/TollheitsTobi/projektSeminar">Simon Lausch, Jan Felix Fuchs & Paul Jansen haben sich mit Deep Q-Learning auseinander gesetzt.</a></center>

# Grober PLan
1. Einleitung in das Projekt
    - Was ist Reinforcement Learning (Agent, Environment, Reward function)
    - Welchen Ansatz verwende ich und warum?
    - Fragestellung für die Ergebnisse vom Projekt
2. Environment
    - Gym
    - Freeway
    - Observation
    - Reward
3. Model Ensemble
    - Daten Sammeln
    - Prepocessing
    - Neuronales Netzwerk
    - Training und Testen
    - Custom Env
4. ME-TRPO
    - Wieso werden welche Parameter verwendet?
5. Ergebnisse
    - Tabelle aller Ergebnisse

## 1. Einleitung in das Projekt

### 1.1 Reinforcement Learning
Reinforcement learning ist eine von drei machine learning Arten, neben supervised-und unsupervised learning. Alle reinforcement learning Ansätze haben die selben Grundbausteine und zwar: ein Environment, einen Agent, states, rewards und actions. Der Agent ist in einem state und muss eine action ausführen, das Environment gibt ihn dafür einen reward und einen neuen state zurück, so versucht der Agent den kumulativen reward zu maximieren. Man kann reinforcement learning noch weiter in model-free und model-based unterteilen, in model-free Ansätzen trainiert der Agent im richtigen Environment, während in model-based Ansätzen ein model des Environments erstellt und als simulation des echten Enironment benutzt wird. Man hat also den Vorteil das man wenn man zum Beispiel einen Roboter trainieren möchte die Zeit im realen Environment minimiert und so auch die chance für mögliche Hardware schäden minimiert.
Irgendwas zur reward function

### 1.2 Welcher Ansatz?
Wir haben uns in der Gruppe so aufgeteilt das Simon Lausch, Jan Felix Fuchs und Paul Jansen einen model-free Ansatz machen und ich einen model-based Ansatz. Normaler model-based Ansatz:
1. Daten im echten Environment Sammeln
2. Model mit den Daten trainieren
3. Policy mit Backpropagation through time trainieren
4. Wiederhole 3 bis es aufhört sich zu verbessern
5. Wiederhole 1-4 bis es funktioniert

Mein Ansatz ME-TRPO (Model Ensemble Trust Region Policy Optimization) hat zwei große unterschiede. Der Erste ist, dass man nicht ein model sondern mehrere models trainiert und als simulation des Environment verwendet. Der zweite unterschied ist, dass man nicht den Backpropagation through time Algorithmus zum trainieren sondern den Trust Region Policy Opimization Algorithmus verwendet.
Der Vorteil von ME ist, dass es besser generalisiert als einzelne models und dadurch eine robustere Vorhersage liefern kann. TRPO optimiert die policy vom Agent, indem er sie in kleinen Schritten anpasst und sicherstellt, dass diese Schritte innerhalb einer festgelegten Trust Region bleiben. Das führt zu einer stabileren optimisierung und verhindert, dass sich die policy des Agents plötzlich stark verändert, was zu instabilen Verhaltensweisen führen könnte.

### 1.3 Projektziel
In diesem Projekt untersuche ich was eine geeignete Anzahl an models ist und wie viele Schritte ME-TRPO weniger als ein model-free Ansatz benötigt.

In [1]:
import gym
from torch import nn
import torch
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import random
import numpy as np
from sb3_contrib import TRPO
from gym import spaces
import os
from torch.utils.data import Dataset
import time

## 2. Environment

### 2.1 Gym
Gym ist eine Open-Source-Library für Reinforcement Learning, die von OpenAI entwickelt wurde. Sie bietet viele verschiedene Environments an, die alle die selbe Struktur haben, so kann man sehr schnell viele verschiedene Environments benutzen. Eigene Environments kann man auch erstellen und so ist es möglich models für ME-TRPO einfach in ein Environment zu machen. Es gibt viele verschiedene libraries die auf gym aufbauen und die dokumentation ist auch sehr gut.

### 2.2 Freeway
Als Environment in dem wir den Agent trainieren haben wir das Atari Spiel Freeway gewählt, in dem es darum geht das ein Huhn eine Straße mit insgesamt zehn Spuren so oft wie möglich in 2:30 Mintuen überquert. Je Spur fährt ein Auto mit unterschiedlichen Geschwindigkeiten aber immer in die selbe Richtung. Die einzigen actions die der Agent ausführen kann sind: nichts tun, hoch und runter. Freeway hat zwei Schwierigkeiten, bei der Schwierigkeit 0 wird der Agent wenn er angefahren wird ein paar pixel zurückgesetzt und bei der 1 wird er zum Start gesetzt. Es gibt verschiedene Modi die man spielen kann, wir verwenden Schwierigkeit 1 und Mode 3.

### 2.3 Observation
Freeway hat drei verschiedene Arten von Daten die man aus dem Environment bekommen kann.
1. RGB-Array:
2. Grayscaled:
3. RAM: 128 Bytes aus dem Spiel

Ich benutze den RAM als Observation, aber ich verwende nicht alle 128 Byte, weil viele unnötige Daten dabei sind und ich einige auch nicht verstehe. Es werden die y Koordinate des Hühnchen, der Score, der Cooldown und die x Koordinaten der Autos benutzt. Die y Koordinate des Hühnchen liegt zwischen 6 und 176 wobei 6 der Start und 176 das Ziel ist. Der Cooldown wird nur in zwei Situationen gebraucht und zwar wenn das Huhn angefahren wird oder ins Ziel läuft. Die x Koordinaten der Autos liegen zwischen 0 und 159.

In [2]:
class ObservationRAM(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Observation space muss am Anfang festgelegt werden
        self.observation_space = spaces.Box(low=0, high=210, shape=(13, ))

    def observation(self, obs):
        # Die Daten die benutzt werden (y, score, cooldown, auto x pos)
        obs = obs[[14, 103, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117]].astype(float)

        # normalisieren
        obs[0] /= 176
        obs[2] /= 142
        for i in range(3, 13):
            obs[i] /= 160

        return obs

### 2.4 Reward
Eine gute Reward Function zu finden war schwer, weil der Agent entweder immer stehen geblieben oder nach oben gerannt ist. Wenn der Agent einfach nur nach oben rennt ohne auf Autos zu achten bekommt er ungefähr 12 Punkte. Das Ziel ist also eine Reward Function die den Agent fürs stehenbleiben und in Autos laufen bestraft. Bei meiner Reward Function bekommt der Agent wenn er ins Ziel läuft 10.000 Reward, wenn er in ein Auto läuft -1.000 und dann noch jeden step $\frac{3}{2}y-90$.

In [3]:
class LinearReward(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env

    def reward(self, reward):
        # RAM vom env um den reward zu berechnen
        ram = self.env.unwrapped.ale.getRAM()
        reward = 0
        # Wenn man ins Ziel läuft, ist der Cooldown 140 oder 141
        if 140 <= ram[106] <= 141:
            reward += 10000
        # reward für die y Position
        reward += (ram[14] * 1.5) - 90
        # Wenn man überfahren wird, ist der Cooldown zwischen 90 und 100
        if 90 <= ram[106] <= 100:
            reward -= 1000
        return reward

## 3. Model Ensemble

### 3.1 Model des Environment
Es gibt verschiedene Arten von models die man benutzten kann, zum Beispiel Gaussian process, Generalized method of moments oder Neuronales Netzwerk. Da dieses Projekt für das Projekt Seminar deep learning ist, werde ich Neuronale Netzwerke verwenden. Der Input ist der State und eine Action und der Output ist der neue State, also $f(S, A) = S'$. Das Neuronale Netzwerk hat 14 Input Neuronen, einen hidden layer mit 256 Neuronen und 13 Output Neuronen.

In [4]:
class EnvNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(14, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 13),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

### 3.2 Daten sammeln
Die Daten werden in dem normalen Freeway Environment in meinen Wrapper Klassen gesammelt. Der Agent soll die Daten selber sammeln, weil man dann genau die States trainiert in denen der Agent selber ist. In der Ersten Epoche hat der Agent noch nicht genug Daten um eigene Entscheidungen zu treffen, dafür habe ich eine Methode mit einem Bias Richtung nach oben gehen implementiert. Die gesammelten Daten werden in einem Dataframe gespeichert und der Dataframe wird zurückgegeben.

In [5]:
def get_action_sample():
    # Random Actions wobei nach oben laufen am Wahrscheinlichsten ist
    x = random.randint(0, 101)
    if x < 90:
        return 1
    if x < 97:
        return 2
    return 0

def get_env_data(n):
    # Warten bis das Model zum Daten sammeln gespeichert wurde
    while len(os.listdir("game_model/trpo3/")) == 0:
        time.sleep(1)

    env_data = LinearReward(ObservationRAM(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="rgb_array", difficulty=1, mode=3)))
    model = TRPO.load("game_model/trpo3/tmp", env=env_data)

    observation = env_data.reset()
    df = pd.DataFrame(observation).T
    # Actions: 0: nichts, 1: up, 2: down
    actions = []

    for i in range(n):
        # model bestimmt action
        action = model.predict(observation)

        # in den ersten x läufen generiert das model keine actions und es wird auf die get_action_smaple methode zurückgegriffen
        if isinstance(action, tuple):
            action = get_action_sample()

        # observation 0 wird vor der schleife gespeichert
        if i != 0:
            df.loc[len(df)] = observation
        actions.append(action)
        observation, reward, done, info = env_data.step(action)
        if done:
            observation = env_data.reset()
    env_data.close()

    # df wird mit den richtigen Daten fertiggestellt
    df["actions"] = actions
    df = df.rename(columns={0: "y", 1: "score", 2: "cooldown",
                            3: "car1", 4: "car2", 5: "car3", 6: "car4", 7: "car5", 8: "car6", 9: "car7", 10: "car8", 11: "car9", 12: "car10"})

    return df

### 3.3 Daten verarbeiten
Um die Neuronalen Netzte richtig trainieren zu können brauchen wir noch einen Dataframe mit den Ergebnissen. In dem Dataframe df ist der State und die Action und im dfY der nächste State.

In [6]:
def process_dfs(df):
    df = df.tail(len(df) - 1)

    dfY = df.copy()
    dfY.drop(["actions"], axis=1, inplace=True)

    dfY = dfY.drop(dfY.index[[0]])
    df = df.drop(df.index[[len(df) - 1]])
    dfY.index = df.index

    df = df.reset_index(drop=True)
    dfY = dfY.reset_index(drop=True)
    return df, dfY

### 3.4 Training und Testen
Loss function: L1Loss
Optimizer: Adam
earlystopping: 10
Ganzer Batch wird auf den GPU gespeichert

In [7]:
def train(X, y, model, loss_fn, optimizer, batch_size):
    model.train()
    loss_sum = 0

    for i in range(round((len(y) / batch_size) + 0)):
        # In jedem durchlauf wird der ganze batch auf dem gpu gespeichert
        train_X = torch.from_numpy(X[i * batch_size:(i + 1) * batch_size]).cuda()
        train_y = torch.from_numpy(y[i * batch_size:(i + 1) * batch_size]).cuda()

        loss = 0

        # loss wird für jeden Datensatz berechnet
        for k in range(min(batch_size, len(train_X))):
            pred = model.forward(train_X[k].float())
            loss += loss_fn(pred.to(torch.float32), train_y[k].to(torch.float32))
        loss_sum += loss.item()

        # bp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # return: avg loss
    return loss_sum / len(y)


def test(X, y, model, loss_fn, batch_size):
    loss_sum = 0

    model.eval()
    with torch.no_grad():
        for i in range(round((len(y) / batch_size) + 1)):
            # In jedem durchlauf wird der ganze batch auf dem gpu gespeichert
            test_X = torch.from_numpy(X[i * batch_size:(i + 1) * batch_size]).cuda()
            test_y = torch.from_numpy(y[i * batch_size:(i + 1) * batch_size]).cuda()

            # loss wird für jeden Datensatz berechnet
            for k in range(min(batch_size, len(test_X))):
                pred = model.forward(test_X[k].float())
                loss = loss_fn(pred, test_y[k])
                loss_sum += loss.item()

    loss_sum /= len(y)
    print(f"Avg loss: {loss_sum}!")
    return loss_sum

def train_test_model(X, y, batch_size, learning_rate):
    # X: input, y: target
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

    # Reset parameters
    model = EnvNetwork().cuda()
    for layer in model.children():
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    last_test_avg = 10000
    overfit = 0
    epochs = 200

    # Alle Dataframes zu numpy arrays
    train_X = train_X.to_numpy()
    test_X = test_X.to_numpy()
    train_y = train_y.to_numpy()
    test_y = test_y.to_numpy()

    for t in range(epochs):
        train_l = train(train_X, train_y, model, loss_fn, optimizer, batch_size)
        test_avg = test(test_X, test_y, model, loss_fn, batch_size)

        # Wenn der Test avg schlechter als vorher ist, wird der last_test_avg nicht aktualisiert
        # und der overfit counter erhöht. Falls der counter >= 10 wird das training frühzeitig abgebrochen
        if test_avg > last_test_avg:
            overfit += 1
        else:
            overfit = 0
            last_test_avg = test_avg
        if overfit >= 10:
            break
    return model

### 3.5 Custom Environment
Ein Custom Environment von Gym braucht immer eine init, step und reset Methoden, in der init muss der Observation und Action Space initialisiert werden. In der step Methode werden die Neuronalen Netze bentutzt in dem wir ein random model aus der list nehmen und damit den nächsten State bekommen. In der step Methode wird wird der Reward berechnet und die reset Methode wird das Environment in den Anfangszustand zurückgesetzt.

In [12]:
class CustomEnv(gym.Env):
    def __init__(self, models):
        # n models
        self.models = models
        self.GAME_START = [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        # state ist am anfang der start punkt von jedem spiel
        self.state = self.GAME_START.copy()
        self.score = 0
        # observation und action space müssen vorher festgelegt werden
        # observation space: y, score, cooldown, auto x
        self.observation_space = spaces.Box(low=0, high=210, shape=(13, ))
        # action space: 1, 2, 3
        self.action_space = spaces.Discrete(3)
        self.step_counter = 0
        self.done = False

    def step(self, action):
        # normalisieren
        self.state[0] /= 176
        self.state[2] /= 142
        for i in range(3, 13):
            self.state[i] /= 160
        self.done = False
        # state wird um action erweitert
        self.state.append(action)
        self.state = torch.tensor(self.state).cuda()
        # eins der n models predictet den nächsten state
        self.state = self.models[np.random.randint(0, len(self.models))].forward(self.state.float())
        self.state = self.state.tolist()

        # Daten wieder in die richtige form
        self.state[0] *= 176
        self.state[2] *= 142
        for i in range(3, 13):
            self.state[i] *= 160

        # state runden
        self.state = [round(i) for i in self.state]

        # gleiche Reward function wie nin der LinearReward class
        reward = (self.state[0] * 1.5) - 90
        if self.state[0] == 0:
            self.state[0] += 0.1
        if 90 <= self.state[2] <= 100:
            reward -= 10000
        if self.state[1] - 0.5 > self.score or self.state[0] >= 176:
            self.score = self.state[1]
            reward += 10000
        self.step_counter += 1
        # Falls 2048 steps gemacht wurden, wird das spiel zurückgesetzt
        if self.step_counter % 2048 == 0:
            self.state = self.GAME_START.copy()
            self.done = True
        return self.state, reward, self.done, {}

    def reset(self):
        self.state = self.GAME_START.copy()
        self.score = 0
        return self.state

## ME-TRPO
Jede Epoche werden 4096 Samples gesammelt, dass sind genau zwei ganze Durchläufe.

In [ ]:
%%time

N_DATA = 8192
N_MODELS = 5
ITERATIONS_PER_EPOCH = 200
STEPS_PER_ITERATION = 2048
EPOCHS = 25
trpo = TRPO("MlpPolicy", LinearReward(ObservationRAM(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="rgb_array", difficulty=1, mode=3))), gamma=0.99, verbose=1)
#trpo = TRPO.load("game_model/models/model_4", env=LinearReward(ObservationRAM(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="rgb_array", difficulty=1, mode=3))))
trpo.save("game_model/trpo3/tmp")

data = []
#data.append(pd.read_csv("game_model/data/data.csv").drop(["Unnamed: 0"], axis=1))
scores = []

def make_models(n):
    df = get_env_data(N_DATA)

    scores.append(max(df["score"]))

    data.append(df)
    df = pd.concat(data)
    df.to_csv("game_model/data/data3.csv")

    df, dfY = process_dfs(df)

    models = []

    for i in range(n):
        models.append(train_test_model(df, dfY, learning_rate=0.0014335761400056032, batch_size=234))
    return models

for i in range(EPOCHS):
    models = make_models(N_MODELS)
    env = CustomEnv(models)

    while len(os.listdir("game_model/trpo3/")) == 0:
        time.sleep(1)

    trpo = TRPO.load("game_model/trpo3/tmp", env=env)
    os.remove("game_model/trpo3/tmp.zip")
    while len(os.listdir("game_model/trpo3/")) == 1:
        time.sleep(1)

    print(f"Epoche: {i}\n________________________________________________________")
    trpo.learn(total_timesteps=ITERATIONS_PER_EPOCH*STEPS_PER_ITERATION, log_interval=25)
    trpo.save(f"game_model/trpo3/tmp")
    trpo.save(f"game_model/models/model3_{i}")

trpo.save(f"trpo_models/model_3")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Avg loss: 0.06761843649569614!
Avg loss: 0.04763362886326999!
Avg loss: 0.03785652707219054!
Avg loss: 0.016775329922315674!
Avg loss: 0.014816545065762455!
Avg loss: 0.013693206132799847!
Avg loss: 0.012836588676433752!
Avg loss: 0.012178255970040297!
Avg loss: 0.011466561438368734!
Avg loss: 0.010789033259537083!
Avg loss: 0.010122357734302753!
Avg loss: 0.009445200085523284!
Avg loss: 0.008762717920426152!
Avg loss: 0.008160233493245713!
Avg loss: 0.007610488294190001!
Avg loss: 0.007118725060347324!
Avg loss: 0.0066481641912302936!
Avg loss: 0.006198371833982735!
Avg loss: 0.005878606788424399!
Avg loss: 0.005541345140560189!
Avg loss: 0.005197254449586912!
Avg loss: 0.004942117388639474!
Avg loss: 0.004715436192535758!
Avg loss: 0.0045088013498095604!
Avg loss: 0.004354714539799162!
Avg loss: 0.00422610457320324